In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import os
import cv2
import importlib
import torch
import numpy as np
import argparse
import yaml
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageOps
import json
import nltk
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn.functional as F
from scipy import signal
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import clip
import kornia
import torchvision
import skimage.color
import random

from utils_func import *
from html_images import *

C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

# Quant

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--device', type=str, default='cuda:0')

parser.add_argument('--root_dir', type=str, default='C:/MyFiles/CondTran/finals/quant')
parser.add_argument('--log_dir', type=str, default='logs/bert_quant')
parser.add_argument('--step', type=str, default='142124')

# Load transformer
args = parser.parse_args(args=[])
os.chdir(args.root_dir)
sys.path.append(args.root_dir)
module = importlib.import_module(f'filltran.models.colorization')
args.fill_model = getattr(module, 'Colorization')
filltran = load_model(args.fill_model, args.log_dir, args.step).to(args.device).eval().requires_grad_(False)

In [ ]:
img_path = 'C:\\MyFiles\\CondTran\\data\\raw\\in1.JPEG'
I_color = Image.open(img_path).convert('RGB')
I_gray = I_color.convert('L')
idx_shape = [I_color.size[1]//16, I_color.size[0]//16]

x_color = preprocess(I_color, None).to(args.device)
x_gray = preprocess(I_gray, None).to(args.device)

# Recon with original color indices
'''idx_color, idx_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
idx_gray = idx_gray.reshape([idx_gray.shape[0]]+idx_shape)
dec = filltran.hybrid_vqgan.decode(idx_color, idx_gray)
dec = output_to_pil(dec[0])
dec.show()'''

# Recon with full color indices
x_color[:, 0, :, :] = 0.0; x_color[:, 1, :, :] = -0.6; x_color[:, 2, :, :] = -0.6
idx_color, idx_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
idx_gray = idx_gray.reshape([idx_gray.shape[0]]+idx_shape)
dec = filltran.hybrid_vqgan.decode(idx_color, idx_gray)
output_to_pil(dec[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\recon_r.png')
output_to_pil(x_color[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\r.png')

# Recon with full color indices
x_color[:, 0, :, :] = -0.6; x_color[:, 1, :, :] = 0.0; x_color[:, 2, :, :] = -0.6
idx_color, idx_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
idx_gray = idx_gray.reshape([idx_gray.shape[0]]+idx_shape)
dec = filltran.hybrid_vqgan.decode(idx_color, idx_gray)
output_to_pil(dec[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\recon_g.png')
output_to_pil(x_color[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\g.png')

# Recon with full color indices
x_color[:, 0, :, :] = -0.6; x_color[:, 1, :, :] = -0.6; x_color[:, 2, :, :] = 0.0
idx_color, idx_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
idx_gray = idx_gray.reshape([idx_gray.shape[0]]+idx_shape)
dec = filltran.hybrid_vqgan.decode(idx_color, idx_gray)
output_to_pil(dec[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\recon_b.png')
output_to_pil(x_color[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\b.png')

# Recon with full gray gray indices
'''I_allgray = 128 * np.ones(list(I_color.size[::-1])).astype(np.uint8)
I_allgray = Image.fromarray(I_allgray)
x_allgray = preprocess(I_allgray, None).to(args.device)
idx_color, idx_allgray = filltran.hybrid_vqgan.encode(x_color, x_allgray)
idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
idx_allgray = idx_allgray.reshape([idx_allgray.shape[0]]+idx_shape)
dec = filltran.hybrid_vqgan.decode(idx_color, idx_allgray)
dec = output_to_pil(dec[0])
dec.show()'''

Gray reconstruction

In [ ]:
img_path = 'C:\\MyFiles\\Dataset\\imagenet\\val5000\\val'
save_path = 'C:\\Users\\lucky\\Desktop\\ablation\\quant_gray'
img_size = [256, 256]

files = os.listdir(img_path)

for f in tqdm(files):
    fname = f.split('.')[0]
    I_color = Image.open(os.path.join(img_path, f)).convert('RGB').resize(img_size)
    I_gray = I_color.convert('L')
    idx_shape = [I_color.size[1]//16, I_color.size[0]//16]

    x_color = preprocess(I_color, None).to(args.device)
    x_gray = preprocess(I_gray, None).to(args.device)

    # Recon with original color indices
    idx_color, idx_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
    idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
    idx_gray = idx_gray.reshape([idx_gray.shape[0]]+idx_shape)
    dec_gt = filltran.hybrid_vqgan.decode(idx_color, idx_gray)
    dec_gt = output_to_pil(dec_gt[0])

    # Recon with full red color indices
    I_rand = np.random.randint(0, 256, size=[1, 1, 3], dtype=np.uint8)
    I_rand = I_rand.repeat(I_color.size[1], axis=0).repeat(I_color.size[0], axis=1)
    I_rand = Image.fromarray(I_rand)
    x_rand = preprocess(I_rand, None).to(args.device)
    idx_rand, _ = filltran.hybrid_vqgan.encode(x_rand, None)
    idx_rand = idx_rand.reshape([idx_rand.shape[0]]+idx_shape)
    dec_rand = filltran.hybrid_vqgan.decode(idx_rand, idx_gray)
    dec_rand = output_to_pil(dec_rand[0])

    I_gray.convert('RGB').save( os.path.join(save_path, 'gray', f'{fname}.png') )
    dec_gt.convert('L').convert('RGB').save( os.path.join(save_path, 'gt', f'{fname}.png') )
    dec_rand.convert('L').convert('RGB').save( os.path.join(save_path, 'rand', f'{fname}.png') )

Color reconstructions

In [ ]:
img_path = 'C:\\MyFiles\\Dataset\\imagenet\\val5000\\val'
save_path = 'C:\\Users\\lucky\\Desktop\\ablation\\quant_recon'
img_size = [256, 256]

files = os.listdir(img_path)

for f in tqdm(files):
    fname = f.split('.')[0]
    I_color = Image.open(os.path.join(img_path, f)).convert('RGB').resize(img_size)
    I_gray = I_color.convert('L')
    idx_shape = [I_color.size[1]//16, I_color.size[0]//16]

    x_color = preprocess(I_color, None).to(args.device)
    x_gray = preprocess(I_gray, None).to(args.device)

    # Recon with original color indices
    idx_color, idx_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
    idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
    idx_gray = idx_gray.reshape([idx_gray.shape[0]]+idx_shape)
    dec_gt = filltran.hybrid_vqgan.decode(idx_color, idx_gray)
    dec_gt = output_to_pil(dec_gt[0])

    I_color.convert('RGB').save( os.path.join(save_path, 'original', f'{fname}.png') )
    dec_gt.convert('RGB').save( os.path.join(save_path, 'recon', f'{fname}.png') )

# Final

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--device', type=str, default='cuda:0')

parser.add_argument('--root_dir', type=str, default='C:/MyFiles/CondTran/finals/bert_final')
parser.add_argument('--log_dir', type=str, default='logs/bert')
parser.add_argument('--step', type=str, default='142124')

# Load transformer
args = parser.parse_args(args=[])
os.chdir(args.root_dir)
sys.path.append(args.root_dir)
module = importlib.import_module(f'filltran.models.colorization')
args.fill_model = getattr(module, 'Colorization')
filltran = load_model(args.fill_model, args.log_dir, args.step).to(args.device).eval().requires_grad_(False)

C:\Users\lucky\Anaconda3\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
C:\Users\lucky\Anaconda3\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


Working with z of shape (1, 1024, 16, 16) = 262144 dimensions.


In [ ]:
img_path = 'C:\\MyFiles\\CondTran\\data\\raw\\in1.JPEG'
I_color = Image.open(img_path).convert('RGB')
I_gray = I_color.convert('L')
idx_shape = [I_color.size[1]//16, I_color.size[0]//16]

x_color = preprocess(I_color, None).to(args.device)
x_gray = preprocess(I_gray, None).to(args.device)

# Recon with original color indices
idx_color, f_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
dec = filltran.hybrid_vqgan.decode(idx_color, f_gray)
dec = output_to_pil(dec[0])
dec.show()

# Recon with full red color indices
'''I_allcolor = 128 * np.ones(list(I_color.size[::-1])+[3]).astype(np.uint8)
I_allcolor[:, :, 1:3] = 50
I_allcolor = Image.fromarray(I_allcolor)
x_allcolor = preprocess(I_allcolor, None).to(args.device)
idx_allcolor, f_gray = filltran.hybrid_vqgan.encode(x_allcolor, x_gray)
idx_allcolor = idx_allcolor.reshape([idx_allcolor.shape[0]]+idx_shape)
dec_gray = filltran.hybrid_vqgan.decode(idx_allcolor, f_gray)
dec_gray = output_to_pil(dec_gray[0])
dec_gray.show()'''

# Recon with full gray gray indices
'''I_allgray = 128 * np.ones(list(I_color.size[::-1])).astype(np.uint8)
I_allgray = Image.fromarray(I_allgray)
x_allgray = preprocess(I_allgray, None).to(args.device)
idx_color, f_allgray = filltran.hybrid_vqgan.encode(x_color, x_allgray)
idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
dec = filltran.hybrid_vqgan.decode(idx_color, f_allgray)
dec = output_to_pil(dec[0])
dec.show()'''

Gray reconstruction

In [ ]:
img_path = 'C:\\MyFiles\\Dataset\\imagenet\\val5000\\val'
save_path = 'C:\\Users\\lucky\\Desktop\\ablation\\chroma_gray'
img_size = [256, 256]

files = os.listdir(img_path)

for f in tqdm(files):
    fname = f.split('.')[0]
    I_color = Image.open(os.path.join(img_path, f)).convert('RGB').resize(img_size)
    I_gray = I_color.convert('L')
    idx_shape = [I_color.size[1]//16, I_color.size[0]//16]

    x_color = preprocess(I_color, None).to(args.device)
    x_gray = preprocess(I_gray, None).to(args.device)

    # Recon with original color indices
    idx_color, f_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
    idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
    dec_gt = filltran.hybrid_vqgan.decode(idx_color, f_gray)
    dec_gt = output_to_pil(dec_gt[0])

    # Recon with full red color indices
    I_rand = np.random.randint(0, 256, size=[1, 1, 3], dtype=np.uint8)
    I_rand = I_rand.repeat(I_color.size[1], axis=0).repeat(I_color.size[0], axis=1)
    I_rand = Image.fromarray(I_rand)
    x_rand = preprocess(I_rand, None).to(args.device)
    idx_rand, _ = filltran.hybrid_vqgan.encode(x_rand, None)
    idx_rand = idx_rand.reshape([idx_rand.shape[0]]+idx_shape)
    dec_rand = filltran.hybrid_vqgan.decode(idx_rand, f_gray)
    dec_rand = output_to_pil(dec_rand[0])

    I_gray.convert('RGB').save( os.path.join(save_path, 'gray', f'{fname}.png') )
    dec_gt.convert('L').convert('RGB').save( os.path.join(save_path, 'gt', f'{fname}.png') )
    dec_rand.convert('L').convert('RGB').save( os.path.join(save_path, 'rand', f'{fname}.png') )

Color reconstructions

In [3]:
img_path = 'C:\\MyFiles\\Dataset\\imagenet\\val5000\\val'
save_path = 'C:\\Users\\lucky\\Desktop\\ablation\\chroma_recon'
img_size = [256, 256]

files = os.listdir(img_path)

for f in tqdm(files):
    fname = f.split('.')[0]
    I_color = Image.open(os.path.join(img_path, f)).convert('RGB').resize(img_size)
    I_gray = I_color.convert('L')
    idx_shape = [I_color.size[1]//16, I_color.size[0]//16]

    x_color = preprocess(I_color, None).to(args.device)
    x_gray = preprocess(I_gray, None).to(args.device)

    # Recon with original color indices
    idx_color, f_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
    idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
    dec_gt = filltran.hybrid_vqgan.decode(idx_color, f_gray)
    dec_gt = output_to_pil(dec_gt[0])

    I_color.convert('RGB').save( os.path.join(save_path, 'original', f'{fname}.png') )
    dec_gt.convert('RGB').save( os.path.join(save_path, 'recon', f'{fname}.png') )

100%|██████████| 5000/5000 [22:01<00:00,  3.79it/s]


# Gray comparison

In [ ]:
def rgb_to_L(x):
    x = np.array(x)
    lab = cv2.cvtColor(x, cv2.COLOR_RGB2LAB)
    l = lab[:, :, 0]

    return Image.fromarray(l)

In [ ]:
img_path = 'C:\\MyFiles\\Dataset\\imagenet\\val5000\\val'
img_size = [256, 256]

rec_loss = []
dec_loss = []
lab_loss = []

files = os.listdir(img_path)[:10]

for f in tqdm(files):
    I_color = Image.open(os.path.join(img_path, f)).convert('RGB').resize(img_size)
    I_gray = rgb_to_L(I_color)
    idx_shape = [I_color.size[1]//16, I_color.size[0]//16]

    x_color = preprocess(I_color, None).to(args.device)
    x_gray = preprocess(I_gray, None).to(args.device)

    # Recon with original color indices
    idx_color, f_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
    idx_color = idx_color.reshape([idx_color.shape[0]]+idx_shape)
    dec = filltran.hybrid_vqgan.decode(idx_color, f_gray)
    dec = output_to_pil(dec[0])
    #dec.show()

    # Recon with full red color indices
    I_allcolor = 128 * np.ones(list(I_color.size[::-1])+[3]).astype(np.uint8)
    I_allcolor[:, :, 1:3] = 50
    I_allcolor = Image.fromarray(I_allcolor)
    x_allcolor = preprocess(I_allcolor, None).to(args.device)
    idx_allcolor, _ = filltran.hybrid_vqgan.encode(x_allcolor, None)
    idx_allcolor = idx_allcolor.reshape([idx_allcolor.shape[0]]+idx_shape)
    dec_gray = filltran.hybrid_vqgan.decode(idx_allcolor, f_gray)
    dec_gray = output_to_pil(dec_gray[0])
    #dec_gray.show()

    # LAB
    dec_lab = draw_color(I_color, I_allcolor, [None, None, None, None])
    #dec_lab.show()

    g = np.array(I_gray)
    g_rec = np.array(rgb_to_L(dec_gray))
    g_lab = np.array(rgb_to_L(dec_lab))
    g_dec = np.array(rgb_to_L(dec))

    rec_loss.append( np.sqrt(np.mean(np.square(g_rec - g))) )
    dec_loss.append( np.sqrt(np.mean(np.square(g_dec - g))) )
    lab_loss.append( np.sqrt(np.mean(np.square(g_lab - g))) )

print(np.mean(rec_loss))
print(np.mean(dec_loss))
print(np.mean(lab_loss))